In [1]:
# --------------------
# 라이브러리 및 데이터 준비
# --------------------
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor # LOF 함수

# 두 개의 밀도가 다른 군집 생성
rng = np.random.RandomState(42)

# 군집 A: 밀도가 높음 (X_a)
X_a = 0.3 * rng.randn(60, 2)
X_a = X_a + 2

# 군집 B: 밀도가 낮음 (X_b)
X_b = 0.8 * rng.randn(40, 2)
X_b = X_b - 2

# 이상치 생성 (전체 데이터에서 멀리 떨어진 지점)
X_outliers = np.array([[5, 5], [-5, 5]])

# 데이터 통합
X_data = np.r_[X_a, X_b, X_outliers]

df = pd.DataFrame(X_data, columns=['Feature1', 'Feature2'])
print("--- 원본 데이터셋 (고밀도, 저밀도 군집 + 이상치) ---")
print(df.head())

--- 원본 데이터셋 (고밀도, 저밀도 군집 + 이상치) ---
   Feature1  Feature2
0  2.149014  1.958521
1  2.194307  2.456909
2  1.929754  1.929759
3  2.473764  2.230230
4  1.859158  2.162768


In [2]:
# --------------------
# 2단계: LOF 모델 학습 및 예측
# --------------------
# n_neighbors=20: 국소 밀도를 계산할 때 주변 20개의 이웃을 사용
# contamination=0.02: 이상치 비율 추정 (총 102개 중 2개, 약 2%)
model = LocalOutlierFactor(n_neighbors=20, contamination=0.02)

# fit_predict 사용: LOF 모델은 주로 이상치 점수(LOF 값)를 계산하는 데 사용되며,
#                   별도의 .transform() 없이 바로 예측(-1 또는 1) 결과를 반환합니다.
# 결과: -1 (이상치), 1 (정상)
predictions = model.fit_predict(X_data)

df['anomaly_prediction'] = predictions

print("\n--- 2단계: 이상치 예측 완료 (1=정상, -1=이상치) ---")
print(df.tail()) # 이상치 데이터는 -1로 예측됨


--- 2단계: 이상치 예측 완료 (1=정상, -1=이상치) ---
     Feature1  Feature2  anomaly_prediction
97  -1.861455 -1.691746                   1
98  -2.707086 -1.877020                   1
99  -1.953433 -2.914376                   1
100  5.000000  5.000000                  -1
101 -5.000000  5.000000                  -1


In [ ]:

# # Local Outlier Factor (LOF) 이상치 탐지 결과 해석

# # --- 2단계: 이상치 예측 완료 (1=정상, -1=이상치) ---
# # LOF는 주변 이웃과의 밀도를 비교하여, 주변보다 밀도가 현저히 낮은 데이터를 이상치로 분류합니다.

# # Column 해석:
# # Feature1, Feature2: 모델 입력으로 사용된 두 개의 특성(좌표) 값입니다.
# # anomaly_prediction: LOF 모델이 이 데이터 포인트를 분류한 결과입니다.
# #                     - **1:** 정상 (주변 밀도와 비슷하거나 높음)
# #                     - **-1:** 이상치 (주변 밀도보다 현저히 낮음)

#      Feature1  Feature2  anomaly_prediction
# 97  -1.861455 -1.691746                   1 # 군집 B (저밀도)에 속하는 데이터. 주변에 이웃이 충분하므로 정상(1)으로 분류됨.
# 98  -2.707086 -1.877020                   1 # 군집 B 내의 데이터. 주변 밀도가 낮더라도 국소적으로는 이웃이 있으므로 정상(1)으로 분류됨.
# 99  -1.953433 -2.914376                   1 # 군집 B 내의 데이터. 정상(1)으로 분류됨.
# 100  5.000000  5.000000                  -1 # 사전에 생성한 명확한 이상치. 주변 20개 이웃과의 밀도가 극도로 낮으므로 **이상치(-1)**로 정확히 분류됨.
# 101 -5.000000  5.000000                  -1 # 사전에 생성한 또 다른 명확한 이상치. 주변 이웃이 거의 없어 **이상치(-1)**로 정확히 분류됨.

# # 요약: LOF는 데이터가 고밀도 군집에 있든 저밀도 군집에 있든, 그 '국소적'인 밀도를 기반으로 이상치를 잘 분리해냅니다.


In [3]:
# --------------------
# 3단계: LOF 점수 (Negative Outlier Factor) 확인 및 해석
# --------------------
# NOF (Negative Outlier Factor) 확인: LOF 값의 음수 역수
# 이 값이 낮을수록 (음수 폭이 클수록) LOF 값이 크다는 의미이므로 이상치 확률이 높음.
negative_outlier_factors = model.negative_outlier_factor_
df['anomaly_score (NOF)'] = negative_outlier_factors

# 탐지된 이상치 확인 (점수(NOF)가 낮은 순서로 정렬)
outliers_check = df.sort_values(by='anomaly_prediction', ascending=True)

print("\n--- 3단계: 탐지된 이상치 (Prediction=-1 & NOF가 가장 낮음) ---")
# 예측이 -1이고, NOF 점수가 가장 낮은 데이터가 가장 확실한 이상치
print(outliers_check[outliers_check['anomaly_prediction'] == -1])

# 해석: LOF 값 자체가 높을수록 이상치이지만, Scikit-learn은 비교를 위해 NOF(음수 역수)를 반환합니다.
#       따라서 NOF 점수가 가장 낮은(가장 음수인) 데이터가 가장 강력한 이상치입니다.


--- 3단계: 탐지된 이상치 (Prediction=-1 & NOF가 가장 낮음) ---
     Feature1  Feature2  anomaly_prediction  anomaly_score (NOF)
101 -5.000000  5.000000                  -1            -7.487925
37   1.214076  2.246571                  -1            -1.916386
100  5.000000  5.000000                  -1            -9.961947
